In [1]:
import datetime
import itertools
import numpy as np
from qiskit import Aer, IBMQ
from qiskit.providers.ibmq import IBMQBackend, IBMQJob
from mzm_generation import (
    FermionicGaussianStateParameters,
    FermionicGaussianStateTask,
    MeasurementErrorCalibrationTask,
    measurement_pauli_strings,
    run_fermionic_gaussian_state_task,
    run_measurement_error_calibration_task,
    run_measurement_error_correction,
    save
)

In [2]:
# get backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
hardware_backend = provider.get_backend('ibmq_manila')
simulator_backend = Aer.get_backend("statevector_simulator")

In [3]:
# set parameters

# backend
backend = simulator_backend

# number of modes
n_modes = 3

# Hamiltonian parameters to consider
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = np.linspace(1e-8, 3.1, num=5)

# qubits
# TODO choose qubits that minimize error rate
qubits = [0, 1, 2]

# number of shots
shots = 10000

# number of shots for measurement error calibration
measurement_error_calibration_shots = 50000

# experiment id used for saving data
experiment_id = f'{datetime.datetime.now().isoformat()}_{backend.name()}'

In [4]:
# generate tasks
tasks = []
for tunneling, superconducting, chemical_potential in itertools.product(
        tunneling_values, superconducting_values, chemical_potential_values):
    for n_particles in range(n_modes + 1):
        for occupied_orbitals in itertools.combinations(range(n_modes), n_particles):
            state_params = FermionicGaussianStateParameters(
                n_modes=n_modes,
                tunneling=tunneling,
                superconducting=superconducting,
                chemical_potential=chemical_potential,
                occupied_orbitals=occupied_orbitals)
            task = FermionicGaussianStateTask(
                experiment_id=experiment_id,
                shots=shots,
                state_params=state_params)
            tasks.append(task)

In [5]:
# run measurement error calibration
measurement_error_calibration_task = MeasurementErrorCalibrationTask(experiment_id, shots=measurement_error_calibration_shots)
run_measurement_error_calibration_task(measurement_error_calibration_task, backend, qubits)

In [6]:
# schedule jobs
# TODO save job information so they can be retrieved from provider later
jobs = []
for task in tasks:
    job = run_fermionic_gaussian_state_task(task, backend, qubits)
    jobs.append((task, job))

In [7]:
# save results
for task, job in jobs:
    circuit_names = [circuit.name for circuit in job.circuits()]
    measurements = dict(zip(circuit_names, job.result().get_counts()))
    data = {
        'measurements': measurements
    }
    save(task, data)

In [8]:
%%time
# run measurement error mitigation and save results
for task in tasks:
    run_measurement_error_correction(measurement_error_calibration_task, task, qubits)

In [3]:
%%time
# test cell to rerun error mitigation

backend = hardware_backend
n_modes = 3
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = np.linspace(1e-8, 3.1, num=20)
qubits = [0, 1, 2]
shots = 10000
measurement_error_calibration_shots = 50000
experiment_id = '2022-01-11T16:50:30.417884_ibmq_manila'
measurement_error_calibration_task = MeasurementErrorCalibrationTask(experiment_id, shots=measurement_error_calibration_shots)
tasks = []
for tunneling, superconducting, chemical_potential in itertools.product(
        tunneling_values, superconducting_values, chemical_potential_values):
    for n_particles in range(n_modes + 1):
        for occupied_orbitals in itertools.combinations(range(n_modes), n_particles):
            state_params = FermionicGaussianStateParameters(
                n_modes=n_modes,
                tunneling=tunneling,
                superconducting=superconducting,
                chemical_potential=chemical_potential,
                occupied_orbitals=occupied_orbitals)
            task = FermionicGaussianStateTask(
                experiment_id=experiment_id,
                shots=shots,
                state_params=state_params)
            tasks.append(task)
for task in tasks:
    run_measurement_error_correction(measurement_error_calibration_task, task, qubits)

CPU times: user 54.1 s, sys: 55.7 ms, total: 54.1 s
Wall time: 2.53 s
